In [39]:
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import array_contains, array_except,explode,explode_outer,lit , regexp_extract

In [6]:
spark=SparkSession.builder.master('local').appName('Testing Fns').getOrCreate()

In [12]:
# Creating a DataFrame
df = spark.createDataFrame([(["a", "b", "c"],), ([],)], ['data'])
df.select(array_contains(df.data,"a")).show()

+-----------------------+
|array_contains(data, a)|
+-----------------------+
|                   true|
|                  false|
+-----------------------+



In [20]:
df=spark.createDataFrame([Row(c1=["b","c","a"],c2=["c","d","a","f"])])
print(df.show())
df.select(array_except(df.c1,df.c2)).show()

+---------+------------+
|       c1|          c2|
+---------+------------+
|[b, c, a]|[c, d, a, f]|
+---------+------------+

None
+--------------------+
|array_except(c1, c2)|
+--------------------+
|                 [b]|
+--------------------+



In [26]:
# Understanding explode
eDF=spark.createDataFrame([Row(a=1,intlist=[1,2,3],mapfield={"a":"b"})])
eDF.show()

+---+---------+--------+
|  a|  intlist|mapfield|
+---+---------+--------+
|  1|[1, 2, 3]|[a -> b]|
+---+---------+--------+



In [27]:
eDF.select(explode(eDF.mapfield)).show()

+---+-----+
|key|value|
+---+-----+
|  a|    b|
+---+-----+



In [47]:
df=spark.createDataFrame([(1,['a','b'],{"x":1.0}),(2,[],{}),(3,None,None)],['id','an_array','a_map'])
df.show()

+---+--------+----------+
| id|an_array|     a_map|
+---+--------+----------+
|  1|  [a, b]|[x -> 1.0]|
|  2|      []|        []|
|  3|    null|      null|
+---+--------+----------+



In [32]:
df.select(explode_outer(df.a_map)).show()

+----+-----+
| key|value|
+----+-----+
|   x|  1.0|
|null| null|
|null| null|
+----+-----+



In [34]:
# Unlike explode_outer
df.select(explode(df.a_map)).show()

+---+-----+
|key|value|
+---+-----+
|  x|  1.0|
+---+-----+



In [48]:
df=df.withColumn("Constant",lit(2))

In [49]:
df.dtypes

[('id', 'bigint'),
 ('an_array', 'array<string>'),
 ('a_map', 'map<string,double>'),
 ('Constant', 'int')]

In [52]:
df.select(regexp_extract("Constant",r"(\d)+",1).alias("Pattern_Match")).show()

+-------------+
|Pattern_Match|
+-------------+
|            2|
|            2|
|            2|
+-------------+



In [ ]:
df.persist()